<a href="https://colab.research.google.com/github/Sakinat-Folorunso/OOU_CSC309_Artificial_Intelligence/blob/main/notebooks/CSC309_Week10_Vision_Robot_Student_Centred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC309 – Artificial Intelligence  
**Week 10 Lab:** Computer Vision (Face Detection) & Simple Robot Programming

**Instructor:** Dr Sakinat Folorunso

**Title:** Associate Professor of AI Systems and FAIR Data **Department:** Computer Sciences, Olabisi Onabanjo University, Ago-Iwoye, Ogun State, Nigeria

**Course Code:** CSC 309

**Mode:** Student‑centred, hands‑on in Google Colab

> Every code cell is commented line‑by‑line so you can follow the logic precisely.

## How to use this notebook
1. Start with the **Group Log** and **Do Now**.  
2. Run the **Setup** cell once.  
3. Work through **Tasks**. Edit only cells marked **`# TODO(Student)`**.  
4. Use **Quick Checks** to test your understanding.  
5. Finish with the **Reflection**. If you finish early, try the **Extensions**.

In [ ]:
#@title 🧑🏽‍🤝‍🧑🏾 Group Log (fill before you start)
# The '#@param' annotations create form fields in Colab for easy input.

group_members = "Type names here"  #@param {type:"string"}  # Names of teammates
roles_notes = "Driver/Navigator, decisions, questions"  #@param {type:"string"}  # Short working notes

print("👥 Group:", group_members)        # Echo the group list for confirmation
print("📝 Notes:", roles_notes)          # Echo the notes so they're preserved in output

### Learning Objectives
- Run a basic **face detection** pipeline (Haar cascade).  
- Control a simple **grid robot** using sensor readings.

In [ ]:
#@title 🔧 Setup
# OpenCV for vision; NumPy + Matplotlib for arrays/plots.

import sys, subprocess                                         # For installs
def pip_install(pkgs):
    for p in pkgs:
        try: __import__(p.split("==")[0])                      # Try import
        except Exception:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", p])
pip_install(["opencv-python-headless", "numpy", "matplotlib"]) # Install needed libraries

import cv2                                                     # Computer vision library
import numpy as np                                             # Arrays and grids
import matplotlib.pyplot as plt                                # Plotting
import os, urllib.request                                      # File handling and download utilities

print("✅ Setup complete for Week 10.")

In [ ]:
#@title 🙂 Face detection demo (fully commented)

# Download Haar cascade file for face detection if missing
if not os.path.exists("haarcascade_frontalface_default.xml"):           # Check if cascade file exists
    url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"
    urllib.request.urlretrieve(url, "haarcascade_frontalface_default.xml")  # Download the file

# Download a sample image if missing
if not os.path.exists("people.jpg"):                                    # Check if demo image exists
    url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg"
    urllib.request.urlretrieve(url, "people.jpg")                        # Download demo image

img = cv2.imread("people.jpg")                                          # Read the image from disk (BGR format)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                            # Convert to grayscale for detection
cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")  # Load the pre‑trained Haar cascade
faces = cascade.detectMultiScale(gray, 1.1, 4)                          # Run the detector (scale=1.1, neighbors=4)

for (x, y, w, h) in faces:                                              # Loop through detected face rectangles
    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)              # Draw a rectangle on each face

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))                        # Convert BGR→RGB for Matplotlib
plt.axis("off")                                                         # Hide axes for cleaner display
plt.title(f"Detected faces: {len(faces)}")                              # Title showing the count
plt.show()                                                              # Display the result

In [ ]:
#@title 🤖 Simple grid robot (fully commented)

class RobotWorld:
    def __init__(self, n=10, obstacles=15, seed=0):
        np.random.seed(seed)                          # Fix NumPy RNG for reproducibility
        self.n = n                                    # Grid dimension (n x n)
        self.grid = np.zeros((n, n), dtype=int)       # 0 = free, 1 = obstacle
        for _ in range(obstacles):                    # Place a number of obstacles at random
            x, y = np.random.randint(0, n), np.random.randint(0, n)
            self.grid[x, y] = 1
        self.start = (0, 0)                           # Start position
        self.goal = (n-1, n-1)                        # Goal position
        self.grid[self.start] = 0                     # Ensure start is free
        self.grid[self.goal] = 0                      # Ensure goal is free
        self.pos = self.start                         # Current robot position

    def sensors(self):
        x, y = self.pos                               # Unpack current coordinates
        g = self.grid; n = self.n                     # Short aliases
        return {                                      # Booleans for whether a move is blocked
            "UP":    (x == 0     or g[x-1, y] == 1),
            "DOWN":  (x == n-1   or g[x+1, y] == 1),
            "LEFT":  (y == 0     or g[x, y-1] == 1),
            "RIGHT": (y == n-1   or g[x, y+1] == 1),
        }

    def act(self, a):
        x, y = self.pos                               # Current coordinates
        s = self.sensors()                            # Read sensors to check for blocks
        if a == "UP" and not s["UP"]:       x -= 1    # Move up if not blocked
        if a == "DOWN" and not s["DOWN"]:   x += 1    # Move down if not blocked
        if a == "LEFT" and not s["LEFT"]:   y -= 1    # Move left if not blocked
        if a == "RIGHT" and not s["RIGHT"]: y += 1    # Move right if not blocked
        self.pos = (x, y)                             # Update position
        return self.pos                               # Return new position

world = RobotWorld(seed=1)                            # Create a world with a fixed seed
print("Start:", world.start, "Goal:", world.goal, "Sensors:", world.sensors())  # Quick status print

In [ ]:
# TODO(Student): Implement a simple controller to reach the goal.
# Hint 1: Use a wall‑following strategy with 'sensors()' to avoid obstacles.
# Hint 2: Or, import your A* function from Week 4 and plan a path on 'world.grid'.